# 零件白斑缺陷检测系统使用教程

本教程将指导您如何使用优化后的零件白斑缺陷检测系统进行图像检测和实时摄像头检测。

## 1. 环境准备

首先导入必要的库并设置工作目录：

In [ ]:
import cv2
import numpy as np
from IPython.display import Image, display
import os
import yaml
from detect import detect_defects_yolo, run_live_detection

# 设置项目根目录
ROOT_DIR = 'D:\\Program\\defect_detection'

# 创建结果目录
os.makedirs(os.path.join(ROOT_DIR, 'results'), exist_ok=True)

# 加载配置文件
def load_config():
    config_path = os.path.join(ROOT_DIR, 'configs', 'config.yaml')
    with open(config_path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

# 全局配置
CONFIG = load_config()

## 2. 单张图片检测

我们可以使用训练好的模型对单张图片进行缺陷检测。首先检查所需文件是否存在：

In [ ]:
def check_files_exist(image_path, weights_path):
    """检查必要文件是否存在
    
    Args:
        image_path: 图像路径
        weights_path: 模型权重路径
    Returns:
        bool: 是否所有文件都存在
    """
    if not os.path.exists(image_path):
        print(f"错误：找不到测试图片: {image_path}")
        validation_dir = os.path.dirname(image_path)
        if os.path.exists(validation_dir):
            print("\n可用的验证集图片:")
            for f in os.listdir(validation_dir)[:5]:
                print(f"- {f}")
        return False
        
    if not os.path.exists(weights_path):
        print(f"错误：找不到模型权重: {weights_path}")
        for pt_file in os.listdir(ROOT_DIR):
            if pt_file.endswith('.pt'):
                print(f"找到可用的权重文件: {pt_file}")
                return False
    return True

# 设置路径
image_path = os.path.join(ROOT_DIR, 'data', 'NEU-DET', 'validation', 'images', 'patches_241.jpg')
weights_path = os.path.join(ROOT_DIR, 'runs', 'detect', 'train2', 'weights', 'best.pt')
save_path = os.path.join(ROOT_DIR, 'results', 'demo_result.jpg')

if check_files_exist(image_path, weights_path):
    # 运行检测
    result = detect_defects_yolo(
        image_path=image_path,
        model_path=weights_path,
        conf_thres=CONFIG['detect']['conf_thres'],
        save_path=save_path
    )
    
    # 显示结果
    if os.path.exists(save_path):
        display(Image(filename=save_path))
    else:
        print(f"错误：结果图片未生成: {save_path}")

## 3. 批量图片检测

我们可以对文件夹中的多张图片进行批量检测：

In [ ]:
def batch_detect(input_dir, output_dir, weights_path, conf_thres=None):
    """批量检测文件夹中的图片
    
    Args:
        input_dir: 输入图片目录
        output_dir: 输出结果目录
        weights_path: 模型权重路径
        conf_thres: 置信度阈值，如果为None则使用配置文件中的值
    """
    if not os.path.exists(input_dir):
        print(f"错误：输入目录不存在: {input_dir}")
        return
        
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(input_dir) 
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not image_files:
        print(f"警告：在 {input_dir} 中没有找到图片文件")
        return
        
    print(f"找到 {len(image_files)} 个图片文件")
    
    # 使用配置文件中的置信度阈值
    if conf_thres is None:
        conf_thres = CONFIG['detect']['conf_thres']
    
    for image_file in image_files:
        input_path = os.path.join(input_dir, image_file)
        output_path = os.path.join(output_dir, f"result_{image_file}")
        
        try:
            detect_defects_yolo(
                image_path=input_path,
                model_path=weights_path,
                conf_thres=conf_thres,
                save_path=output_path
            )
            print(f'处理完成: {image_file}')
        except Exception as e:
            print(f'处理 {image_file} 时出错: {str(e)}')

input_dir = os.path.join(ROOT_DIR, 'data', 'NEU-DET', 'validation', 'images')
output_dir = os.path.join(ROOT_DIR, 'results', 'batch_results')

if os.path.exists(weights_path):
    batch_detect(input_dir, output_dir, weights_path)

## 4. 实时摄像头检测

如果您的电脑有摄像头，可以进行实时检测：

In [ ]:
def start_live_detection(weights_path=None, save_dir=None):
    """启动实时摄像头检测
    
    Args:
        weights_path: 模型权重路径，如果为None则使用默认路径
        save_dir: 保存视频的目录，如果为None则不保存
    """
    if weights_path is None:
        weights_path = os.path.join(ROOT_DIR, 'models', 'best.pt')
        
    if not os.path.exists(weights_path):
        print(f"错误：找不到模型权重: {weights_path}")
        return
        
    try:
        run_live_detection(weights=weights_path, save_dir=save_dir)
    except Exception as e:
        print(f"启动实时检测时出错: {str(e)}")

# 使用示例
save_dir = os.path.join(ROOT_DIR, 'results', 'live_detection')
start_live_detection(weights_path=weights_path, save_dir=save_dir)